Como o arquivo é baixado é muito grande para subir no Git apagamos e deixamos para que o usuario possa fazer o download.

In [2]:
from download_dados import ColigacaoDownloader 

cd = ColigacaoDownloader()
cd.download(['2022'])
cd.unzip(['2022'], 'SP')

Descompactando zipfile: 100%|██████████| 1/1 [00:00<00:00, 48.68it/s]


In [3]:
from download_dados import CandidatoDownloader 

cd = CandidatoDownloader()
cd.download(['2022'])
cd.unzip(['2022'], 'SP')

Descompactando zipfile: 100%|██████████| 1/1 [00:00<00:00, 35.48it/s]


In [31]:
import os

def save_csv(arq_nome, cargo, uf, ano, df, turno=None):
    if turno:
        path = os.path.join('dados', 'processados', uf, ano, cargo, turno)
    else:
        path = os.path.join('dados', 'processados', uf, ano, cargo)

    if not os.path.exists(path):
        os.makedirs(path)
    
    path = os.path.join(path, f'{arq_nome}.csv')
    df.to_csv(path)

In [32]:
import pandas as pd

df_coligacao = pd.read_csv('dados/brutos/SP/2022/consulta_coligacao_2022_SP.csv', encoding='latin-1', sep=';')
df_coligacao.drop(['DT_GERACAO', 'HH_GERACAO', 'ANO_ELEICAO', 'CD_TIPO_ELEICAO', 'NM_TIPO_ELEICAO', 'CD_ELEICAO', 'DS_ELEICAO', 'DT_ELEICAO', 'SG_UF', 'SG_UE'], axis=1, inplace=True)

df_coligacao_df = df_coligacao[(df_coligacao['CD_CARGO']==6)]
save_csv('coligacao', 'DeputadoFederal', 'SP', '2022', df_coligacao_df)

df_coligacao_de = df_coligacao[(df_coligacao['CD_CARGO']==7)]
save_csv('coligacao', 'DeputadoEstadual', 'SP', '2022', df_coligacao_de)

In [4]:
import pandas as pd

df_cand = pd.read_csv('dados/brutos/SP/2022/consulta_cand_2022_SP.csv', encoding='latin-1', sep=';')

In [71]:
import pandas as pd

df_votos_secao = pd.read_csv('dados/brutos/SP/2022/votacao_secao_2022_SP.csv', encoding='latin-1', sep=';')

df_votos_partido_de = df_votos_secao[(df_votos_secao['SQ_CANDIDATO']==-3) & (df_votos_secao['CD_CARGO']==7)]
df_votos_partido_de = df_votos_partido_de.groupby(['SQ_CANDIDATO', 'NM_VOTAVEL', 'NR_VOTAVEL']).agg({'QT_VOTOS':'sum'})
save_csv('votacao_partido', 'DeputadoEstadual', 'SP', '2022', df_votos_partido_de)


df_votos_candidato_df = df_votos_secao[(~df_votos_secao['SQ_CANDIDATO'].isin([-3,-1])) & (df_votos_secao['CD_CARGO']==6)]
df_votos_candidato_df = df_votos_candidato_df.groupby(['SQ_CANDIDATO',  'NM_VOTAVEL', 'NR_VOTAVEL']).agg({'QT_VOTOS':'sum'})
save_csv('votacao_candidato', 'DeputadoFederal', 'SP', '2022', df_votos_candidato_df)


df_votos_candidato_de = df_votos_secao[(~df_votos_secao['SQ_CANDIDATO'].isin([-3,-1])) & (df_votos_secao['CD_CARGO']==7)]
df_votos_candidato_de = df_votos_candidato_de.groupby(['SQ_CANDIDATO', 'NM_VOTAVEL', 'NR_VOTAVEL']).agg({'QT_VOTOS':'sum'})
save_csv('votacao_candidato', 'DeputadoEstadual', 'SP', '2022', df_votos_candidato_de)

df_votos_candidato_gv1 = df_votos_secao[(df_votos_secao['CD_CARGO']==3) & (df_votos_secao['NR_TURNO']==1)]
df_votos_candidato_gv1 = df_votos_candidato_gv1.groupby(['SQ_CANDIDATO', 'NM_VOTAVEL']).agg({'QT_VOTOS':'sum'})
save_csv('votacao_candidato', 'Governador', 'SP', '2022', df_votos_candidato_gv1, '1_TURNO')

df_votos_candidato_gv2 = df_votos_secao[(df_votos_secao['CD_CARGO']==3) & (df_votos_secao['NR_TURNO']==2)]
df_votos_candidato_gv2 = df_votos_candidato_gv2.groupby(['SQ_CANDIDATO', 'NM_VOTAVEL']).agg({'QT_VOTOS':'sum'})
save_csv('votacao_candidato', 'Governador', 'SP', '2022', df_votos_candidato_gv1, '2_TURNO')